## 数据集压缩文件解压
在 python 中解压缩包的方法
```python
import zipfile
with zipfile.ZipFile("Data/dogs-vs-cats.zip","r") as zip_ref:
    # Extract to storage/ of Gradient (free persistent storage)
    zip_ref.extractall("storage/Data")
```

## 加载数据集的三种情况

pytorch加载数据集主要分以下三种情况：：   
1. 所使用数据集已被集成在pytorch内，如：CIFAR-10，CIFAR-100，MNIST等等。对于这种数据集，可以直接使用pytorch内置函数：`torchvision.datasets.CIFAR100` 来直接加载，比较方便。例如： 

```python
cifar100_training = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
```   

2. 所使用的数据集未被集成在pytorch中，，但文件和目录的构造如下：


```
root/dog/xxx.png
root/dog/xxy.png
root/dog/xxz.png

root/cat/123.png
root/cat/nsdf3.png
root/cat/asd932_.png
```  

> 即要求数据文件按照其所属的类/class都有自己的目录(例如 it's own directory `cat` and `dog`)，那么就可以通过torchvision中的通用数据集`ImageFolder`来完成加载。
则这个方法加载时**会使用从目录名称中获取的类来标记(label)图像**   
> + 例如 图像`123.png`将 be loaded with the class label `cat`.   


3. 对于最普通的数据集, 既不是自带数据集，又不满足ImageFolder, 这种时候就自己进行处理: 一般过程为：
> 具体见：https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
  + 首先，**定义数据集的类（myDataset），这个类要继承`dataset`这个抽象类**，并实现`__len__`以及`__getitem__`这两个函数，通常情况还包括初始函数`__init__`.
  + 然后，实现用于特定图像预处理的功能，并封装成类。当然常用的一些变换可以在torchvision中找到。用torchvision.transforms.Compose将它们进行组合成(transform)
  + transform作为上面myDataset类的参数传入，并得到实例化myDataset得到（transformed_dataset）对象。
  + 最后，将transformed_dataset作为torch.utils.data.DataLoader类的形参，并根据需求设置自己是否需要打乱顺序，批大小...


---   
## （二）使用 ImageFolder 读取数据   
对于自定义数据集pytorch实际上是有一个函数的：`torchvision.datasets.ImageFolder()`，此函数只能加载**特定形式**的数据集（图片已被分类好，并放在相应文件夹下了，其标签就是其上层目录的名称）。

### 看看torchvision.datasets.ImageFolder   
这个类是怎么写的，主要代码如下，想详细了解的可以看：官方github代码。看懂了ImageFolder这个类，就可以自定义一个你自己的数据读取接口了。



对于`torchvision.datasets` 中有两个不同的类，分别为`DatasetFolder`和`ImageFolder`, `ImageFolder` 是继承自`DatasetFolder`。
下面我们通过源码来看一看folder文件中`DatasetFolder` 和 `ImageFolder` 分别做了些什么   
+ 完整源码来自：[Pytorch 数据加载与数据预处理](https://blog.csdn.net/a940902940902/article/details/82666824?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromBaidu-2.control&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromBaidu-2.control)   
+ https://pytorch.org/docs/stable/_modules/torchvision/datasets/folder.html#ImageFolder   ：略有不同：[源码笔记：](https://blog.csdn.net/qian2213762498/article/details/86659136?utm_medium=distribute.pc_relevant.none-task-blog-baidulandingword-2&spm=1001.2101.3001.4242)

对于`torchvision.datasets.ImageFolder()`, 先看看 `torchvision.datasets`,  `torchvision.datasets` 中有两个不同的类，分别为`DatasetFolder`和 `ImageFolder`, `ImageFolder` 是继承自`DatasetFolder`:

In [ ]:
import torch.utils.data as data
from PIL import Image
import os
import os.path


def has_file_allowed_extension(filename, extensions):  //检查输入是否是规定的扩展名
    """Checks if a file is an allowed extension.

    Args:
        filename (string): path to a file

    Returns:
        bool: True if the filename ends with a known image extension
    """
    filename_lower = filename.lower()
    return any(filename_lower.endswith(ext) for ext in extensions)


def find_classes(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))] //获取root目录下所有的文件夹名称

    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))} //生成类别名称与类别id的对应Dictionary
    return classes, class_to_idx


def make_dataset(dir, class_to_idx, extensions):
    images = []
    dir = os.path.expanduser(dir)// 将~和~user转化为用户目录，对参数中出现~进行处理
    for target in sorted(os.listdir(dir)):
        d = os.path.join(dir, target)
        if not os.path.isdir(d):
            continue

        for root, _, fnames in sorted(os.walk(d)): //os.work包含三个部分，root代表该目录路径 _代表该路径下的文件夹名称集合，fnames代表该路径下的文件名称集合
            for fname in sorted(fnames):
                if has_file_allowed_extension(fname, extensions):
                    path = os.path.join(root, fname)
                    item = (path, class_to_idx[target])
                    images.append(item)    //生成（训练样本图像目录，训练样本所属类别）的元组

    return images   //返回上述元组的列表

#11111111111111111111111
class DatasetFolder(data.Dataset):
    """A generic data loader where the samples are arranged in this way: ::

        root/class_x/xxx.ext
        root/class_x/xxy.ext
        root/class_x/xxz.ext

        root/class_y/123.ext
        root/class_y/nsdf3.ext
        root/class_y/asd932_.ext

    Args:
        root (string): Root directory path.
        loader (callable): A function to load a sample given its path.
        extensions (list[string]): A list of allowed extensions.
        transform (callable, optional): A function/transform that takes in
            a sample and returns a transformed version.
            E.g, ``transforms.RandomCrop`` for images.
        target_transform (callable, optional): A function/transform that takes
            in the target and transforms it.

     Attributes:
        classes (list): List of the class names.
        class_to_idx (dict): Dict with items (class_name, class_index).
        samples (list): List of (sample path, class_index) tuples
    """

    def __init__(self, root, loader, extensions, transform=None, target_transform=None):
        classes, class_to_idx = find_classes(root)
        samples = make_dataset(root, class_to_idx, extensions)
        if len(samples) == 0:
            raise(RuntimeError("Found 0 files in subfolders of: " + root + "\n"
                               "Supported extensions are: " + ",".join(extensions)))

        self.root = root
        self.loader = loader
        self.extensions = extensions

        self.classes = classes
        self.class_to_idx = class_to_idx
        self.samples = samples

        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        """
        根据index获取sample 返回值为（sample，target）元组，同时如果该类输入参数中有transform和target_transform，torchvision.transforms类型的参数时，将获取的元组分别执行transform和target_transform中的数据转换方法。
              Args:
            index (int): Index

        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        """
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target


    def __len__(self):
        return len(self.samples)

    def __repr__(self): //定义输出对象格式 其中和__str__的区别是__repr__无论是print输出还是直接输出对象自身 都是以定义的格式进行输出，而__str__ 只有在print输出的时候会是以定义的格式进行输出
        fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        fmt_str += '    Root Location: {}\n'.format(self.root)
        tmp = '    Transforms (if any): '
        fmt_str += '{0}{1}\n'.format(tmp, self.transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        tmp = '    Target Transforms (if any): '
        fmt_str += '{0}{1}'.format(tmp, self.target_transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        return fmt_str



IMG_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif']


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)


def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)

#2
class ImageFolder(DatasetFolder): 
    """A generic data loader where the images are arranged in this way: ::

        root/dog/xxx.png
        root/dog/xxy.png
        root/dog/xxz.png

        root/cat/123.png
        root/cat/nsdf3.png
        root/cat/asd932_.png

    Args:
        root (string): Root directory path.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        loader (callable, optional): A function to load an image given its path.

     Attributes:
        classes (list): List of the class names.
        class_to_idx (dict): Dict with items (class_name, class_index).
        imgs (list): List of (image path, class_index) tuples
    """
    def __init__(self, root, transform=None, target_transform=None,
                 loader=default_loader):
        super(ImageFolder, self).__init__(root, loader, IMG_EXTENSIONS,
                                          transform=transform,
                                          target_transform=target_transform)
        self.imgs = self.samples


对上述代码的解析：来自[❤Pytorch源码(一)—— 简析torchvision的ImageFolder](https://www.jianshu.com/p/5bb684c4c9fc)  ，包括 

+ `find_classes`
+ ` has_file_allowed_extension`
+ `make_dataset` 

下面三个函数都是加载图像的函数，用于ImageFolder类中   
+ `pil_loader`
+ `accimage_loader`
+ `default_loader`

看起来很复杂，其实非常简单。继承的类是torch.utils.data.Dataset，主要包含三个方法：初始化`__init__`，获取图像`__getitem__`，数据集数量 `__len__`。`__init__`方法中先通过find_classes函数得到分类的类别名（classes）和类别名与数字类别的映射关系字典（class_to_idx）。然后通过make_dataset函数得到imags，这个imags是一个列表，其中每个值是一个tuple，每个tuple包含两个元素：图像路径和标签。剩下的就是一些赋值操作了。在`__getitem__`方法中最重要的就是 img = self.loader(path)这行，表示数据读取，可以从`__init__`方法中看出self.loader采用的是default_loader，这个default_loader的核心就是用python的PIL库的Image模块来读取图像数据。
> [PyTorch学习之路（level2）——自定义数据读取](https://blog.csdn.net/u014380165/article/details/78634829?utm_medium=distribute.pc_relevant_t0.none-task-blog-BlogCommendFromBaidu-1.control&depth_1-utm_source=distribute.pc_relevant_t0.none-task-blog-BlogCommendFromBaidu-1.control)

一个ps：csv存在的意义:Let’s create a dataset class for our face landmarks dataset. We will read the csv in __init__ but leave the reading of images to __getitem__. This is memory efficient because all the images are not stored in the memory at once but read as required.

数据集中 text文件存在的意义[对于所有的训练样本都在一个文件夹中 同时有一个对应的txt文件每一行分别是对应图像的路径以及其所属的类别，可以参照上述class写出对应的加载类](https://blog.csdn.net/a940902940902/article/details/82666824?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromBaidu-2.control&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromBaidu-2.control)

## (三) 自定义数据集读取   

`torchvision.datasets.ImageFolder()`，此函数只能加载**特定形式**的数据集（图片已被分类好，并放在相应文件夹下了，其标签就是其上层目录的名称）。但是有些情况下你的图像数据不是这样维护的，比如一个文件夹下面各个类别的图像数据都有，同时用一个对应的标签文件，比如txt文件来维护图像和标签的对应关系，在这种情况下就不能用torchvision.datasets.ImageFolder来读取数据了，需要自定义一个数据读取接口。这时候，直接使用`ImageFolder`会导致训练结果与预期结果毫无关系，这就需要我们自己重新构造一个类似于`DatasetFolder`类（ImageFolder就继承了DatasetFolder类）的新类别来加载数据集。

首先在PyTorch中和数据读取相关的类基本都要继承一个基类：torch.utils.data.Dataset。然后再改写其中的`__init__`、`__len__`、`__getitem__`等方法即可。

1. **txt文件**维护的标签数据集例子：[PyTorch学习之路（level2）——自定义数据读取](https://blog.csdn.net/u014380165/article/details/78634829?utm_medium=distribute.pc_relevant_t0.none-task-blog-BlogCommendFromBaidu-1.control&depth_1-utm_source=distribute.pc_relevant_t0.none-task-blog-BlogCommendFromBaidu-1.control)   
下面假设img_path是你的图像文件夹，该文件夹下面放了所有图像数据（包括训练和测试），然后txt_path下面放了train.txt和val.txt两个文件，txt文件中每行都是图像路径，tab键，标签。所以下面代码的__init__方法中self.img_name和self.img_label的读取方式就跟你数据的存放方式有关，你可以根据你实际数据的维护方式做调整。__getitem__方法没有做太大改动，依然采用default_loader方法来读取图像。最后在Transform中将每张图像都封装成Tensor。

一个Map式的数据集必须要重写getitem(self, index)、 len(self) 两个内建方法，用来表示从索引到样本的映射(Map)。这样一个数据集dataset，举个例子，当使用dataset[idx]命令时，可以在你的硬盘中读取数据集中第idx张图片以及其标签(如果有的话); len(dataset)则会返回这个数据集的容量。

自定义数据集类的范式大致是这样的：

In [ ]:
class CustomDataset(torch.utils.data.Dataset):#需要继承torch.utils.data.Dataset
    def __init__(self):
        # TODO
        # 1. Initialize file path or list of file names.
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        #这里需要注意的是，第一步：read one data，是一个data point
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0

In [ ]:
class customData(Dataset):
    def __init__(self, img_path, txt_path, dataset = '', data_transforms=None, loader = default_loader):
        with open(txt_path) as input_file:
            lines = input_file.readlines()
            self.img_name = [os.path.join(img_path, line.strip().split('\t')[0]) for line in lines]
            self.img_label = [int(line.strip().split('\t')[-1]) for line in lines]
        self.data_transforms = data_transforms
        self.dataset = dataset
        self.loader = loader

    def __len__(self):
        return len(self.img_name)

    def __getitem__(self, item):
        img_name = self.img_name[item]
        label = self.img_label[item]
        img = self.loader(img_name)

        if self.data_transforms is not None:
            try:
                img = self.data_transforms[self.dataset](img)
            except:
                print("Cannot transform image: {}".format(img_name))
        return img, label

2. **csv 文件维护数据标签的例子**：把读取图片的函数写在__getitem__中： 【完整参考4】6口罩检测(完整).ipynb

3. 读取时划分训练集的例子：https://www.cnblogs.com/picassooo/p/12846617.html

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
image_transform = transforms.Compose([
    transforms.Resize(256),               # 把图片resize为256*256
    transforms.RandomCrop(224),           # 随机裁剪224*224
    transforms.RandomHorizontalFlip(),    # 水平翻转
    transforms.ToTensor(),                # 将图像转为Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])   # 标准化
])
 
class DogVsCatDataset(Dataset):   # 创建一个叫做DogVsCatDataset的Dataset，继承自父类torch.utils.data.Dataset
    def __init__(self, root_dir, train=True, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.img_path = os.listdir(self.root_dir)
        if train:
            self.img_path = list(filter(lambda x: int(x.split('.')[1]) < 10000, self.img_path))    # 划分训练集和验证集
        else:
            self.img_path = list(filter(lambda x: int(x.split('.')[1]) >= 10000, self.img_path))
        self.transform = transform
 
    def __len__(self):
        return len(self.img_path)
 
    def __getitem__(self, idx):
        image = Image.open(os.path.join(self.root_dir, self.img_path[idx]))
        label = 0 if self.img_path[idx].split('.')[0] == 'cat' else 1        # label, 猫为0，狗为1
        if self.transform:
            image = self.transform(image)
        label = torch.from_numpy(np.array([label]))
        return image, label

4. 查看文件目录知道图片是以这种名称保存的`cat.6089.jpg`, 我们现在要**提取出label**: 在文件  【完整参考3】dogs-vs-cats.ipynb 中的例子： 使用自定义数据集   
但是这个例子不好，应该把读取图片的代码卸载 `getitem`部分